**Cargar librerias necesarias**

In [12]:
import numpy as np
import pandas as pd
import tensorflow as tf

**Cargar datos**

In [6]:
df = pd.read_csv('listings_limpio.csv')
df.head()

,id,host_response_rate,host_acceptance_rate,neighbourhood_cleansed,latitude,longitude,accommodates,bathrooms,bedrooms,beds,...,Safe,Refrigerator,Smoke_alarm_home_safety,Essentials,Services,property_Entire_Place,property_Hotel_Room,property_Other,property_Private_Room,property_Shared_Room
0,30320,100,100,Sol,40.41476,-3.70418,2,1.0,1.0,2.0,...,0,0,0,0,0,True,False,False,False,False
1,40916,100,100,Universidad,40.42247,-3.70577,2,1.0,1.0,3.0,...,0,0,0,0,0,True,False,False,False,False
2,62423,100,99,Justicia,40.41884,-3.69655,4,1.5,1.0,2.0,...,0,1,1,1,0,False,False,False,True,False
3,70073,100,100,Universidad,40.42252,-3.70250,2,1.0,1.0,1.0,...,0,1,0,1,1,True,False,False,False,False
4,70310,100,100,Universidad,40.42252,-3.70250,2,1.0,1.0,2.0,...,0,1,0,1,1,True,False,False,False,False


**Separar en train y test**

In [7]:
train = df.sample(frac=0.8, random_state=100)
train.head()

,id,host_response_rate,host_acceptance_rate,neighbourhood_cleansed,latitude,longitude,accommodates,bathrooms,bedrooms,beds,...,Safe,Refrigerator,Smoke_alarm_home_safety,Essentials,Services,property_Entire_Place,property_Hotel_Room,property_Other,property_Private_Room,property_Shared_Room
2415,32892473,100,92,Lucero,40.406490,-3.745560,4,1.0,1.0,1.0,...,0,1,1,1,1,True,False,False,False,False
11422,1183135383050248925,50,40,Universidad,40.421990,-3.702730,3,1.0,1.0,1.0,...,0,0,0,0,0,True,False,False,False,False
13907,1334659658421449266,33,17,Pilar,40.473280,-3.704860,1,1.0,1.0,1.0,...,0,0,0,0,0,False,False,False,True,False
2120,29759801,100,100,San Andrés,40.333614,-3.702798,2,1.0,1.0,2.0,...,0,1,1,1,1,True,False,False,False,False
5370,664313376708004518,99,99,Bellas Vistas,40.447750,-3.708190,4,1.0,1.0,2.0,...,0,1,0,1,1,True,False,False,False,False


In [8]:
test = df.drop(train.index)
test.head()

,id,host_response_rate,host_acceptance_rate,neighbourhood_cleansed,latitude,longitude,accommodates,bathrooms,bedrooms,beds,...,Safe,Refrigerator,Smoke_alarm_home_safety,Essentials,Services,property_Entire_Place,property_Hotel_Room,property_Other,property_Private_Room,property_Shared_Room
26,339171,100,100,Cortes,40.41285,-3.69553,4,1.0,1.0,2.0,...,0,1,1,1,1,True,False,False,False,False
27,342095,100,81,Embajadores,40.41015,-3.69797,2,1.0,1.0,2.0,...,0,1,1,1,1,True,False,False,False,False
40,469231,86,95,Palacio,40.41941,-3.70823,2,1.0,1.0,1.0,...,0,1,0,1,1,True,False,False,False,False
45,502918,100,100,Cortes,40.41155,-3.69492,1,1.0,2.0,1.0,...,0,1,1,1,1,False,False,False,True,False
48,510973,83,8,Cortes,40.41412,-3.70057,1,2.0,1.0,3.0,...,0,1,0,1,0,False,False,False,True,False


In [9]:
print(train.shape)
print(test.shape)

(14131, 40)
(3533, 40)


**Separar X y Y**

In [10]:
y_train = train['price']
y_test = test['price']
x_train = train.drop(columns=['price'])
x_test = test.drop(columns=['price'])

**Construir el modelo usando tensorfloww**

capa de normalización